In [1]:
import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import glob
import gzip
import tensorflow as tf
from tensorflow import keras
import time

import matplotlib.pyplot as plt

SEED = 1013
np.random.seed(SEED)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


import pandas as pd 
import numpy as np 
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy import sparse
import os
import pickle
import emoji
from wordcloud import WordCloud

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.util import ngrams

nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Meghna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = '/Users/Meghna/Desktop/tweets_for_language_model_noRT.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
df_for_fasttext = pd.DataFrame()
df

,Unnamed: 0,races,tweets
0,0,0,And on the first day of the Colorado legislatu...
1,1,1,Everyone needs to read this. Dr. Nichols and h...
2,2,0,“He would make outrageous claims like he inven...
3,3,0,@holynext SUGAR AND COVID\nDiabetes is a targe...
4,4,1,I’m all for compromise - just do not give in o...
...,...,...,...
37199,37199,1,"There's great news about Covid vaccines, but w..."
37200,37200,1,@andybudd You're saying this like all travelli...
37201,37201,1,Oh they was beaten or pepper sprayed
37202,37202,1,Maine CDC prepares to distribute COVID-19 vacc...


In [5]:
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner_updated(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(r'\@w+','',stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [6]:
tweets = df['tweets']
cleaned_tweets = []
for tweet in tweets:
    cleaned_tweets.append(tweet_cleaner_updated(tweet))

In [8]:
races = df['races']
races_for_fasttext = []
for race in races:
    if race == 0:
        races_for_fasttext.append('__label__others')
    else:
        races_for_fasttext.append('__label__aa')

In [9]:
df_for_fasttext['races'] = races_for_fasttext
df_for_fasttext['tweets'] = cleaned_tweets

In [10]:
df_for_fasttext

,races,tweets
0,__label__others,and on the first day of the colorado legislatu...
1,__label__aa,everyone needs to read this dr nichols and his...
2,__label__others,he would make outrageous claims like he invent...
3,__label__others,sugar and covid diabetes is target we eat too ...
4,__label__aa,all for compromise just do not give in on the ...
...,...,...
37199,__label__aa,there great news about covid vaccines but why ...
37200,__label__aa,you re saying this like all travelling is non ...
37201,__label__aa,oh they was beaten or pepper sprayed
37202,__label__aa,maine cdc prepares to distribute covid vaccine...


In [11]:
# from sklearn.model_selection import train_test_split

# train, test = train_test_split(df, test_size=0.25, random_state=1)

In [12]:
#train.to_csv(r'/Users/Meghna/Desktop/input.txt', header=None, index=None, sep=' ', mode='a')

In [15]:
#test.to_csv(r'/Users/Meghna/Desktop/validation.txt', header=None, index=None, sep=' ', mode='a')

In [59]:
import fasttext

model = fasttext.train_supervised('training_data.txt', lr=0.25, epoch=150)
model

In [40]:
# print(model.words)
# print(model.labels)

['</s>', 'the', 'to', 'a', 'and', 'of', 'in', 'is', 'for', 'I', 'that', 'you', 'are', 'on', 'this', 'have', 'be', 'with', 'it', 'not', 'from', 'my', 'people', 'as', 'at', 'has', 'but', 'we', 'will', 'vaccine', 'about', 'they', 'your', 'was', 'all', 'so', 'by', '&amp;', 'The', 'get', 'pandemic', 'been', '-', 'who', 'can', 'or', 'COVID', 'just', 'Covid', 'out', 'would', 'if', 'more', 'no', 'do', 'like', 'our', 'their', 'an', 'COVID-19', 'me', 'one', 'had', 'how', 'covid', 'what', 'because', 'up', 'need', 'during', 'when', 'than', 'he', 'there', 'his', 'should', 'know', 'work', 'new', 'being', 'going', '#COVID19', 'some', 'This', 'We', 'only', 'go', 'take', 'help', 'still', '"I', 'mask', 'now', 'were', 'via', 'think', 'many', 'them', 'after', 'time', '"The', 'over', 'lockdown', 'other', 'even', 'could', 'see', 'You', 'us', 'make', 'any', 'back', 'If', "don't", 'really', 'don’t', 'wear', 'virus', 'want', 'those', 'They', 'into', 'coronavirus', 'home', '"This', 'It', 'cases', 'these', 'I’m'

In [44]:
model.save_model("fastText_race_model.bin")

In [45]:
model = fasttext.load_model("fastText_race_model.bin")

In [60]:
model.test('testing_data.txt', k=1)

(6186, 0.9988684125444552, 0.9988684125444552)

In [51]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(3, p))
    print("R@{}\t{:.3f}".format(3, r))

print_results(*model.test('validation.txt'))

N	0
P@3	nan
R@3	nan


In [62]:
# CHECK PERFORMANCE      
result = model.test('training_data.txt')
validation = model.test('testing_data.txt')
        
# DISPLAY ACCURACY OF TRAINED MODEL
#text_line = str(hyper_params) + ",accuracy:" + str(result[1])  + ",validation:" + str(validation[1]) + '\n' 
text_line = ",accuracy:" + str(result[1])  + ",validation:" + str(validation[1]) + '\n' 
print(text_line)

,accuracy:0.9987217251153226,validation:0.9988684125444552

